In [18]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import sklearn
import regex as re
import emoji
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from tensorflow.keras.models import Sequential
from keras import layers
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
import keras
from keras.regularizers import l1, l2
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification, BertForSequenceClassification
import torch

In [19]:
df=pd.read_excel("LargeDataset.xlsx")
df['text'].dropna()
df

,text,label,
0,going through depression open this thread,1,NaN
1,fivebalanceusa good afternoon would honored to...,0,NaN
2,fkin depressed gonna feeling like this,1,NaN
3,when have anxiety every pleasure guilty pleasure,0,NaN
4,sorry canxexxt talk right ixexxm doing girl sh...,1,NaN
...,...,...,...
7693,that frequently,0,NaN
7694,counterargument krugman predicted great depres...,1,NaN
7695,think dignity overrated that status doled limi...,1,NaN
7696,cant make depression disappear over night sure...,0,NaN


In [20]:
#removing emojis
df["text"]=df["text"].astype(str)
df["text"]=df["text"].apply(emoji.demojize)
df

,text,label,
0,going through depression open this thread,1,NaN
1,fivebalanceusa good afternoon would honored to...,0,NaN
2,fkin depressed gonna feeling like this,1,NaN
3,when have anxiety every pleasure guilty pleasure,0,NaN
4,sorry canxexxt talk right ixexxm doing girl sh...,1,NaN
...,...,...,...
7693,that frequently,0,NaN
7694,counterargument krugman predicted great depres...,1,NaN
7695,think dignity overrated that status doled limi...,1,NaN
7696,cant make depression disappear over night sure...,0,NaN


In [21]:
#removing URls
df["text"] = df["text"].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
df

,text,label,
0,going through depression open this thread,1,NaN
1,fivebalanceusa good afternoon would honored to...,0,NaN
2,fkin depressed gonna feeling like this,1,NaN
3,when have anxiety every pleasure guilty pleasure,0,NaN
4,sorry canxexxt talk right ixexxm doing girl sh...,1,NaN
...,...,...,...
7693,that frequently,0,NaN
7694,counterargument krugman predicted great depres...,1,NaN
7695,think dignity overrated that status doled limi...,1,NaN
7696,cant make depression disappear over night sure...,0,NaN


In [22]:
#removing non english words
words = set(nltk.corpus.words.words())
def removeNonEng(sent):
    k=" ".join(w for w in nltk.wordpunct_tokenize(sent) \
    if w.lower() in words or not w.isalpha())
    return k

df["text"]=df["text"].apply(removeNonEng)
df

,text,label,
0,going through depression open this thread,1,NaN
1,good afternoon would took look project,0,NaN
2,depressed feeling like this,1,NaN
3,when have anxiety every pleasure guilty pleasure,0,NaN
4,sorry talk right doing girl laundry depressed,1,NaN
...,...,...,...
7693,that frequently,0,NaN
7694,counterargument great depression march dead sq...,1,NaN
7695,think dignity that status limited supply shame...,1,NaN
7696,cant make depression disappear over night sure...,0,NaN


In [23]:
#removing stopwords
stop_words= stopwords.words('english')
personal_pronouns= ['i', 'you', 'she', 'he', 'they'] #keeping personal pronouns to serve as a feature

for word in personal_pronouns:
    if word in stop_words:
        stop_words.remove(word)

df["text"].apply(lambda x: [item for item in x if item not in stop_words])
df

,text,label,
0,going through depression open this thread,1,NaN
1,good afternoon would took look project,0,NaN
2,depressed feeling like this,1,NaN
3,when have anxiety every pleasure guilty pleasure,0,NaN
4,sorry talk right doing girl laundry depressed,1,NaN
...,...,...,...
7693,that frequently,0,NaN
7694,counterargument great depression march dead sq...,1,NaN
7695,think dignity that status limited supply shame...,1,NaN
7696,cant make depression disappear over night sure...,0,NaN


In [24]:
#vectorizing natural language
X=df["text"]
y=df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [25]:
#models

In [11]:
#logistic regression
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 score:", f1_score(y_test, y_pred))

Accuracy: 0.8963477357706689
Precision: 0.88934034416826
Recall: 0.8555989882731663
F1 score: 0.8721434431032462


In [10]:
#knn
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 score:", f1_score(y_test, y_pred))

Accuracy: 0.9120507685916078
Precision: 0.8107365792759051
Recall: 0.7466084157277535
F1 score: 0.7773521666267657


In [12]:
#decision tree
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 score:", f1_score(y_test, y_pred))

Accuracy: 0.9211593269630245
Precision: 0.8097345132743363
Recall: 0.7994941365831225
F1 score: 0.804581742450538


In [13]:
#random forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 score:", f1_score(y_test, y_pred))

Accuracy: 0.939948815953469
Precision: 0.8739798367738838
Recall: 0.8372039549321683
F1 score: 0.8551967116852613


In [14]:
#svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 score:", f1_score(y_test, y_pred))

Accuracy: 0.9403071873701704
Precision: 0.8759635599159075
Recall: 0.862267187859278
F1 score: 0.8690614136732329


In [9]:
#cnn
from tensorflow.python.keras import regularizers
input_dim = X_train.shape[1]
model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu')) #kernel_regularizer=l2(0.01)
model.add(Dropout(0.2, input_shape=(60,)))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

c:\users\varun sovani\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\users\varun sovani\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\regularization\dropout.py:42: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 20)                  │         278,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 278,741 (1.06 MB)

 Trainable params: 278,741 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=30)

Epoch 1/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8365 - loss: 0.4223 - val_accuracy: 0.9041 - val_loss: 0.2601
Epoch 2/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9235 - loss: 0.2183 - val_accuracy: 0.9113 - val_loss: 0.2381
Epoch 3/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9395 - loss: 0.1727 - val_accuracy: 0.9112 - val_loss: 0.2392
Epoch 4/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9513 - loss: 0.1410 - val_accuracy: 0.9091 - val_loss: 0.2508
Epoch 5/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9578 - loss: 0.1201 - val_accuracy: 0.9103 - val_loss: 0.2596
Epoch 6/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9641 - loss: 0.1059 - val_accuracy: 0.9085 - val_loss: 0.2714
Epoch 7/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9676 - loss: 0.0943 - val_accuracy: 0.9086 - val_loss: 0.2869
Epoch 8/30
1605/1605 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9732 - loss: 0.0828 

In [11]:
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()
y_pred = (y_pred>0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 score:", f1_score(y_test, y_pred))

377/377 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Accuracy: 0.9563024511840465
Precision: 0.9489759171730813
Recall: 0.8973258220280524
F1 score: 0.8980527752502275


In [ ]:
#BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir="./")
encoded_data = tokenizer.batch_encode_plus(df["text"].values, add_special_tokens=True, return_attention_mask=True, pad_to_max_length=True, max_length=256, return_tensors='pt')
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
labels = torch.tensor(df.label.values)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False, cache_dir="./")
batch_size = 32
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
for epoch in range(epochs):
    model.train()
    for i in range(0, input_ids.size(0), batch_size):
        optimizer.zero_grad()
        outputs = model(input_ids[i:i+batch_size], attention_mask=attention_masks[i:i+batch_size], labels=labels[i:i+batch_size])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
    predictions = torch.argmax(outputs[0], dim=1).flatten()
    accuracy = torch.sum(predictions == labels) / len(labels)

print("Accuracy:", accuracy.item())

c:\users\varun sovani\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\users\varun sovani\appdata\local\programs\python\python39\lib\site-packages\transformers\tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in 